# 获得药品的所有相关caseid

In [10]:
!pip install sqlalchemy
!pip install mysql-connector-python
!pip install pymysql
!pip install sqlalchemy

In [36]:
import os
import pymysql
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)

conn = pymysql.connect(
    host='localhost',
    port=int(3306),
    user="root",
    passwd='0921051011Liu',
    db="faers",
    charset='utf8mb4')


In [ ]:
## check data

In [14]:
df = pd.read_sql_query("select count(1),count(distinct primaryid),count(distinct caseid) from faers.faers_demo;", conn)
df

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_8244/2594427308.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("select count(1),count(distinct primaryid),count(distinct caseid) from faers.faers_demo;", conn)


,count(1),count(distinct primaryid),count(distinct caseid)
0,11877243,11877243,10219652


In [12]:
df = pd.read_sql_query("select count(1),count(distinct primaryid),count(distinct caseid) from faers.faers_drug;", conn)
df

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_8244/4200782247.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("select count(1),count(distinct primaryid),count(distinct caseid) from faers.faers_drug;", conn)


,count(1),count(distinct primaryid),count(distinct caseid)
0,17223552,4362113,3895924


In [42]:
sql = ''' 
--   获得药品相关的所有case
DROP TABLE  if exists faers.ribo_drug;
CREATE TABLE faers.ribo_drug as
select primaryid, caseid, DRUG_SEQ, START_DT
from faers.faers_drug d1
where (lower(d1.drugname) like 'ribociclib%'
        OR lower(d1.drugname) like '%kisqali%'
        OR lower(d1.prod_ai) like '%ribociclib%'
        OR lower(d1.prod_ai) like '%kisqali%'
    ) and lower(d1.role_cod) = 'ps'
; '''

df = pd.read_sql_query(sql, conn)
df.tail(10)

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_8244/2129124748.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


DatabaseError: Execution failed on sql ' 
-- 获得药品相关的所有case
DROP TABLE  if exists faers.ribo_caseid_tb;
CREATE TABLE faers.ribo_caseid_tb as
select primaryid, caseid, DRUG_SEQ,START_DT
from faers.faers_drug d1
where (lower(d1.drugname) like 'ribociclib%'
        OR lower(d1.drugname) like '%kisqali%'
        OR lower(d1.prod_ai) like '%ribociclib%'
        OR lower(d1.prod_ai) like '%kisqali%'
    ) and lower(d1.role_cod) = 'ps'
; ': (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'CREATE TABLE faers.ribo_caseid_tb as\nselect primaryid, caseid, DRUG_SEQ,START_DT' at line 3")

In [ ]:

sql = '''
SELECT  sex, COUNT(distinct tb1.caseid) AS caseid_count
FROM
(   select  caseid, max(sex) as sex
    from faers.faers_demo
    group by caseid
) tb1
join (
        select distinct caseid
        from faers.ribo_caseid_tb
) tb2
on tb1.caseid = tb2.caseid
group by sex;
'''

## Table1.1性别

In [ ]:
## select * from faers_reac limit

In [16]:
sql = '''

SELECT  
    case 
        when sex in ('UNK','') then  'Unknown'
        when sex = 'M' then  'Male'
        when sex = 'F' then  'Female'
        else sex
    end as sex, COUNT(distinct tb1.caseid) AS caseid_count
FROM
(   select  caseid, max(sex) as sex
    from faers.faers_demo
    group by caseid
) tb1
join (
        select distinct caseid
        from faers.ribo_drug_usage
) tb2
on tb1.caseid = tb2.caseid
group by 1 order by 1;

'''
df_sex = pd.read_sql_query(sql, conn)
df_sex


/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_8244/2863254186.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sex = pd.read_sql_query(sql, conn)


,sex,caseid_count
0,Female,14061
1,Male,274
2,Unknown,1124


## Table1.2年龄


In [19]:
sql = '''

select
    case
            when age = '无' then 'Unknown'
            when cast(age as signed) >= 0  and cast(age as signed) < 18 then '＜18'
            when cast(age as signed) >= 18 and cast(age as signed) < 29 then '18-29'
            when cast(age as signed) >= 30 and cast(age as signed) < 49 then '30-49'
            when cast(age as signed) >= 50 and cast(age as signed) < 64 then '50-64'
            when cast(age as signed) >= 65 and cast(age as signed) < 74 then '65-74'
            when cast(age as signed) > 75  and cast(age as signed) < 84 then '75-84'
            when cast(age as signed) > 75  and cast(age as signed) < 84 then '75-84'
            else age
    end as 'age',
    COUNT(distinct tb1.caseid) AS caseid_count
FROM
(
    select caseid, max(age) as age
    from faers.faers_demo
    group by caseid
) tb1
join (
        select distinct caseid
        from faers.ribo_drug_usage
) tb2
on tb1.caseid = tb2.caseid
group by 1
order by 1
;
'''
df_age = pd.read_sql_query(sql, conn)
df_age


/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_8244/1031500877.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_age = pd.read_sql_query(sql, conn)


,age,caseid_count
0,＜18,2
1,30-49,1
2,50-64,4
3,65-74,1
4,75-84,1
5,Unknown,15450


## 年龄的分位


In [ ]:
sql = '''

select 
    percentile_approx(age, 0.25) as q25,
    percentile_approx(age, 0.5) as q50,
    percentile_approx(age, 0.75) as q75
FROM
(
    select caseid, max(age) as age
    from faers.faers_demo
    where age != '无'
    group by caseid
) tb1
join (
        select distinct caseid
        from faers.ribo_drug_usage
) tb2
on tb1.caseid = tb2.caseid;

'''

df_age_q = pd.read_sql_query(sql, conn)
df_age_q


# Table1.3体重


## 体重的分桶计


In [ ]:
sql = '''

select
    case
            when wt = '无' then 'Unknown'
            when cast(wt as signed) >= 0  and cast(wt as signed) < 80 then '＜80'
            when cast(wt as signed) >= 80 and cast(wt as signed) < 100 then '80-100'
            when cast(wt as signed) >= 100 then '>100'
            else wt
    end as wt_bucket,
    COUNT(distinct tb1.caseid) AS caseid_count
FROM
(
    select caseid, max(wt) as wt
    from faers.faers_demo
    where upper(I_F_COD) = 'I'
    group by caseid
) tb1
join (
        select distinct caseid
        from faers.ribo_drug_usage
) tb2
on tb1.caseid = tb2.caseid
group by 1
order by 1
;
'''
df_wt = pd.read_sql_query(sql, conn)
df_wt


## 体重的分位点

In [26]:
sql = '''

select 
    percentile(wt, 0.25) as wt_q25,
    percentile(wt, 0.5)  as wt_q50,
    percentile(wt, 0.75) as wt_q75
FROM
(
    select caseid, max(wt) as wt
    from faers.faers_demo
    where wt != '无' and upper(I_F_COD) = 'I'   
    group by caseid
) tb1
join (
        select distinct caseid
        from faers.ribo_drug_usage
) tb2
on tb1.caseid = tb2.caseid;

'''

df_wt_q = pd.read_sql_query(sql, conn)
df_wt_q


/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_8244/36466318.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_wt_q = pd.read_sql_query(sql, conn)


DatabaseError: Execution failed on sql '

select 
    percentile(wt, 0.25) as wt_q25,
    percentile(wt, 0.5)  as wt_q50,
    percentile(wt, 0.75) as wt_q75
FROM
(
    select caseid, max(wt) as wt
    from faers.faers_demo
    where wt != '无' and upper(I_F_COD) = 'I'   
    group by caseid
) tb1
join (
        select distinct caseid
        from faers.ribo_caseid_tb
) tb2
on tb1.caseid = tb2.caseid;

': (1305, 'FUNCTION faers.percentile does not exist')

# Table1.4上报者职业

In [27]:
sql = '''

select
    OCCP_COD,
    COUNT(distinct tb1.caseid) AS caseid_count
FROM
(
    select caseid, max(OCCP_COD) as OCCP_COD
    from faers.faers_demo
    group by caseid
) tb1
join (
        select distinct caseid
        from faers.ribo_drug_usage
) tb2
on tb1.caseid = tb2.caseid
group by 1
order by 1
;
'''
df_reporter = pd.read_sql_query(sql, conn)
df_reporter



/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_8244/3593283355.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_reporter = pd.read_sql_query(sql, conn)


,OCCP_COD,caseid_count
0,,316
1,CN,8032
2,HP,1616
3,MD,3964
4,OT,960
5,PH,571


# Table1.5不良反应造成的结局

In [30]:
sql = '''

select
    OUTC_COD,
    COUNT(distinct tb1.caseid) AS caseid_count
FROM
(
    select caseid, max(OUTC_COD) as OUTC_COD
    from faers.faers_outc
    group by caseid
) tb1
join (
        select distinct caseid
        from faers.ribo_drug_usage
) tb2
on tb1.caseid = tb2.caseid
group by 1
order by 1
;
'''
df_outcome = pd.read_sql_query(sql, conn)
df_outcome



/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_8244/3888994451.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_outcome = pd.read_sql_query(sql, conn)


NameError: name 'df_outcom' is not defined

In [31]:
df_outcome


,OUTC_COD,caseid_count
0,0,2034
1,1,82
2,2,1973
3,3,66
4,4,1
5,5,15
6,6,7270


# Table1.6不良反应上报的国家

In [ ]:
sql = '''

select
    REPORTER_COUNTRY,
    COUNT(distinct tb1.caseid) AS caseid_count
FROM
(
    select caseid, max(REPORTER_COUNTRY) as REPORTER_COUNTRY
    from faers.faers_demo
    group by caseid
) tb1
join (
        select distinct caseid
        from faers.ribo_drug_usage
) tb2
on tb1.caseid = tb2.caseid
group by 1
order by 1
;
'''
df_reporter_country = pd.read_sql_query(sql, conn)
df_reporter_country


/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_44459/2513200929.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_reporter_country = pd.read_sql_query(sql, conn)


# Time of onset 

In [ ]:
## drug usage show

In [38]:
sql ='''
DROP TABLE  if exists faers.drug_usage_with_ae;
CREATE TABLE  faers.drug_usage_with_ae  as
SELECT 
    D.CASEID,
    D.PRIMARYID,
    T.START_DT AS Ribociclib_StartDate,
    D.EVENT_DT AS AdverseEvent_StartDate,
    TIMESTAMPDIFF(DAY, T.START_DT, D.EVENT_DT) AS day_Interval
FROM 
    faers.faers_ther T
JOIN faers.ribo_drug_usage R  -- ribociclib usage
    on  R.PRIMARYID = T.PRIMARYID and T.DSG_DRUG_SEQ = R.DRUG_SEQ
JOIN 
    faers.faers_demo D ON T.PRIMARYID = D.PRIMARYID
WHERE 
    T.START_DT IS NOT NULL  
    AND D.EVENT_DT IS NOT NULL;
'''

df_time_onset = pd.read_sql_query(sql, conn)
df_time_onset

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_8244/367895488.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_time_onset = pd.read_sql_query(sql, conn)


,CASEID,Ribociclib_StartDate,AdverseEvent_StartDate,day_Interval
0,13119314,20161117,20170103.0,47.0
1,13119314,20161117,20170103.0,47.0
2,13119314,20161117,20170103.0,47.0
3,13119314,20161117,20170103.0,47.0
4,13281834,20170126,20170224.0,29.0
...,...,...,...,...
39020,23684450,202112,20230501.0,NaN
39021,23684450,202112,20230501.0,NaN
39022,23685212,20220507,,NaN
39023,23685212,20220507,,NaN


In [ ]:
sql = '''
    
    select 
    if(START_DT IS NOT NULL AND EVENT_DT IS NOT NULL,
        TIMESTAMPDIFF(DAY, T.START_DT, D.EVENT_DT) AS day_Interval,
        'Unkown'
    ) as time_of_onset,
    count(distinct caseid) as caseid_count
    from faers.drug_usage_with_ae
    group by 1
'''
df_time_onset = pd.read_sql_query(sql, conn)
df_time_onset

# Table1.7上报年份

In [ ]:
sql ='''
    SELECT
        YEAR(REPT_DT) AS report_year,
        COUNT(distinct PRIMARYID) AS report_count
    FROM
        faers.faers_demo D
    JOIN 
        faers.ribo_drug_usage R
    ON D.PRIMARYID = R.PRIMARYID
    GROUP BY
        1
    ORDER BY
        1;
'''

df_rpt_year = pd.read_sql_query(sql, conn)
df_rpt_year